In [1]:
import pandas
import numpy

In [2]:
#Cargar datos

In [3]:
ventas=pandas.read_csv(r"D:\Estudiar\MLData\retail.csv")
ventas.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,573744,21314,SMALL GLASS HEART TRINKET POT,8,2011-11-01 08:16:00,2.10,17733.0,United Kingdom
1,573744,21704,BAG 250g SWIRLY MARBLES,12,2011-11-01 08:16:00,0.85,17733.0,United Kingdom
2,573744,21791,VINTAGE HEADS AND TAILS CARD GAME,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom
3,573744,21892,TRADITIONAL WOODEN CATCH CUP GAME,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom
4,573744,21915,RED HARMONICA IN BOX,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom


In [4]:
item_dict=dict(zip(ventas["StockCode"], ventas["Description"]))

In [5]:
item_dict["21314"]

'SMALL GLASS HEART TRINKET POT'

In [6]:
matriz_ventas_df=ventas.pivot_table(
    index="CustomerID",
    columns="StockCode",
    values="Quantity",
    aggfunc="sum"
)

In [8]:
matriz_ventas_df.shape

(1711, 2704)

In [9]:
customer_id_list=numpy.array(matriz_ventas_df.index.to_list())
item_id_list=numpy.array(matriz_ventas_df.columns.to_list())

In [10]:
from scipy.sparse import coo_matrix

In [11]:
ventas_mtz=matriz_ventas_df.fillna(0).values.copy()
ventas_mtz_sparse=coo_matrix(ventas_mtz)

In [12]:
#Descomposicion SVD sparse de la matriz

In [13]:
from scipy.sparse.linalg import svds
U, s, V=svds(ventas_mtz_sparse, k=10)

In [16]:
print("Tamaños")
print(f"U: {U.shape}")
print(f"s: {s.shape}")
print(f"V: {V.shape}")

Tamaños
U: (1711, 10)
s: (10,)
V: (10, 2704)


In [17]:
#Reconstruir la matriz original de cliente/producto con SVD

In [18]:
s_diag=numpy.diag(s)

In [19]:
ventas_svd=U @ s_diag @ V

In [20]:
ventas_svd.shape

(1711, 2704)

In [21]:
#Crear funcion de recomendacion

In [53]:
def recomendar(id_cliente, num_recomendaciones=5):
    #Se toma la fila correspondiente al cliente
    cliente_index=numpy.where(customer_id_list==id_cliente)[0][0]
    #ordenamos las compras predichas por los clientes en valor descendente
    index_sort=ventas_svd[cliente_index, :].argsort()[::-1]
    #creamos una mascara booleana de los productos que no ha comprado el cliente
    productos_no_comprados=ventas_mtz[cliente_index, :][index_sort]==0

    rec_index= index_sort[productos_no_comprados]
    rec_ids= item_id_list[rec_index]
    recomendaciones=rec_ids[:num_recomendaciones]
    return recomendaciones

In [54]:
cliente_id=12352

In [55]:
d=matriz_ventas_df.loc[cliente_id]
[item_dict[x] for x in d[d.notna()].index]

['BLUE STRIPE CERAMIC DRAWER KNOB',
 'VICTORIAN GLASS HANGING T-LIGHT',
 'IVORY KITCHEN SCALES',
 'MINT KITCHEN SCALES',
 'CHILDS BREAKFAST SET DOLLY GIRL ',
 'PINK BABY BUNTING',
 'PANTRY ROLLING PIN',
 'PANTRY PASTRY BRUSH',
 'ZINC HEART FLOWER T-LIGHT HOLDER',
 'GLASS BON BON JAR',
 'PETIT TRAY CHIC',
 'SET 12 COLOUR PENCILS SPACEBOY ',
 'SET 12 COLOUR PENCILS DOLLY GIRL ',
 'WOODLAND BUNNIES LOLLY MAKERS',
 'POSTAGE']

In [56]:
recomendaciones=recomendar(cliente_id)

print(recomendaciones)

['22492' '85099B' '22152' '21810' '22356']
